# KZG Commitments: Simulating a Rollup Sequencer and a Validator in Ethereum (off-chain)

* Ethereum data blob: 128KB per blob.
  * e.g. An Arbitrum batch containing 1,000 transactions → encoded as polynomial coefficients → KZG commitment, proof
  * Prove data availability

* by Team S^quad
* Created on: 9 Augest 2025
* Latest updated: 11 August 2025




## Aim of this notebook

* Explore the role of KZG commitments in Ethereum
* Simulate the validation process of a real data blob on Ethereum

## Before we proceed

### **What is Ethereum?**

* Ethereum is a decentralized, open-source blockchain platform that runs programs (*smart contracts*) on a distributed network of computers, with computation paid for in **gas** (denominated in ETH).

* The Ethereum Virtual Machine (EVM) executes transactions, updates the blockchain’s state (account balances, contract storage, etc.), and enforces the rules of the protocol.

* Types of transactions include:

  * Sending Ether (ETH) between accounts.
  * Calling smart contract functions to interact with decentralized applications.
  * Including special data such as calldata, or large-scale data `blobs` (*EIP-4844*) for scaling rollups.

### **Why is it related to KZG?**

* Ethereum’s recent Dencun upgrade introduced EIP-4844, which adds a new transaction type containing `blobs` — large temporary data chunks used mainly by rollups for scaling.

* Each `blob` is committed to using a KZG (Kate–Zaverucha–Goldberg) polynomial commitment, to produce a small, constant-size fingerprint of the blob’s contents.

* The 48 bytes KZG commitment is stored permanently on Ethereum, while the full blob data 128kB is kept temporarily (\~18 days). This lets Ethereum verify that blob data was correct when posted, without storing huge amount of data forever on-chain.

### **What exactly happens?**

* A User sending a transaction
* A Rollup Sequencer orders them into a batch (sequence) and compress them into a single Ethereum `blob` (EIP-4844), generate a KZG commitment for that `blob` and submit a Type 3 `blob` transaction to Ethereum L1, with a sidecar consists of `blob` data + cryptographic proofs.
* When a validator proposes a block containing a `blob` transaction, they verify the `blob` data and proofs in the sidecar using the KZG proof to confirm the `blob` matches the commitment in the transaction. The blob sidecar is stored in consensus-layer storage for about 18 days before being pruned.

### **How do the validators check a blob?**
* The validator is given the blob data, commitment and proof.
* Using their open-source Ethereum clients, the validator generates the polynomial from the blob data with a fixed chunk size of 32 bytes.
* The validator evaluates the polynomial at a given challenge point 𝑧
to compute a claimed value f(z).
* Finally, using the commitment, the proof, the challenge z, the claimed value f(z), and the public parameter g2_tau, the validator verifies the correctness of the blob.



## Component in this basic Simulation

* Rollup Sequencer (off-chain)
  * Orders transactions and groups them into one or more Ethereum blobs (per EIP-4844).
  * Generates KZG commitments and proofs for each blob along with the blob data.
  * Submits the blob commitments on-chain, along with a sidecar containing the blob data, proofs, and other related information.
* Validator (off-chain)
  * Retrieves the blob data and sidecar.
  * Generates the polynomial representation from the blob data.
  * Uses the given or derived challenge points z and evaluations y.
  * Verifies the blob’s correctness using the commitment, proof, and these values.


### Rollup Sequencer

* #### Have a Ethereum blob ready.

In [169]:
# @title
# Cell 30

blob_hex = "..."

blob_bytes = bytes.fromhex(blob_hex)
print(f"Blob length: {len(blob_bytes)} bytes")  # Should be 131072 bytes

Blob length: 131072 bytes


* #### Generates KZG commitments and proofs with the blob data

In [170]:
# @title
# Cell 05
# Trusted SRS data from the Ethereum KZG Ceremony (ceremony.ethereum.org)
SRS_str = '''
{
    "SRS": {
        "G1Powers": [
                "..."
            ],
        "G2Powers": [
                "0x93e02b6052719f607dacd3a088274f65596bd0d09920b61ab5da61bbdc7f5049334cf11213945d57e5ac7d055d042b7e024aa2b2f08f0a91260805272dc51051c6e47ad4fa403b02b4510b647ae3d1770bac0326a805bbefd48056c8c121bdb8",
                "0xb5bfd7dd8cdeb128843bc287230af38926187075cbfbefa81009a2ce615ac53d2914e5870cb452d2afaaab24f3499f72185cbfee53492714734429b7b38608e23926c911cceceac9a36851477ba4c60b087041de621000edc98edada20c1def2"
            ]
    }
}
'''

print("Done loading a trimmed version of transcript.json.")

Done loading a trimmed version of transcript.json.


In [171]:
# @title
# Cell 06
# KZG Implementation with py_ecc.optimized_bls12_381
# py_ecc Raw code can be found in
#     https://github.com/ethereum/py_ecc/

try:
    installed_py_ecc
except NameError:
    !pip install py_ecc
    installed_py_ecc = 1

# ------------------------------------------------------------
# Run on optimized_bls12_381
# - All our tests are built on this library
# - Use projective coeffs
# - Optimized algorithms no division needed
# - Written in Python + Cython
# - Final exponentiation optimized pairing
# - Faster
# - CONs: Normalization needed for equality checks
# ------------------------------------------------------------

from py_ecc.optimized_bls12_381 import (
    G1 as opt_G1,
    G2 as opt_G2,
    Z1 as opt_Z1,
    Z2 as opt_Z2,
    curve_order as opt_curve_order,
    add as opt_add,
    multiply as opt_multiply,
    neg as opt_neg,
    pairing as opt_pairing,
    normalize as opt_normalize,
    final_exponentiate as opt_final_exponentiate,
    FQ, FQ2, field_modulus
)

from py_ecc.bls.point_compression import compress_G1, compress_G2, decompress_G1, decompress_G2, modular_squareroot_in_FQ2

# ------------------------------------------------------------
# blob to fx
#   - Usage example: blob_to_coeffs(data)
#   - sample data can be found
#     https://etherscan.io/blob/0x014527d555d949b3afcfa246e16eb0e0aef9e9da60b7a0266f1da43b3fd8e8cf?bid=210274
# ------------------------------------------------------------

def blob_to_coeffs(data):
    print("Loaded pre-downloaded blob.")
    data_len = len(data)
    chunk_size = 32
    # print(f"Chunk size: {chunk_size}")
    chunks = [data[i:i+chunk_size] for i in range(0, data_len, chunk_size)]
    return [int.from_bytes(chunk, 'big') % opt_curve_order for chunk in chunks]

# ------------------------------------------------------------
# Polynomial helper methods
#   - Usage example: evaluate_polynomial([a,b,c,d], x)
#     - y = a + bx + cx^2 + dx^3
#     - return y
# ------------------------------------------------------------

def evaluate_polynomial(poly, x):
    """Evaluate polynomial f(x) at x."""
    result = 0
    for coeff in reversed(poly):
        result = (result * x + coeff) % opt_curve_order
    return result

# ------------------------------------------------------------
# Convert commitments and proofs to their hex version
# ------------------------------------------------------------
def int_to_bytes(value: int, length: int) -> bytes:
    return value.to_bytes(length, byteorder='big')

def compress_G1_affine(point: tuple[int, int]):
    x, y = point
    if hasattr(x, 'n'):
        x = x.n
    if hasattr(y, 'n'):
        y = y.n
    x_fq = FQ(x)
    sign_bit = 1 if y > (field_modulus - 1) // 2 else 0
    x_bytes = int_to_bytes(x, 48)
    first_byte = x_bytes[0]
    first_byte |= 0x80
    if sign_bit:
        first_byte |= 0x20

    compressed = bytes([first_byte]) + x_bytes[1:]
    return '0x'+compressed.hex()

def compress_G2_affine(point: tuple[FQ2, FQ2]) -> str:
    x, y = point
    x_c, x_f = x.coeffs  # two FQ elements
    y_c, y_f = y.coeffs
    def fq_to_int(fq):
        return fq.n if hasattr(fq, 'n') else int(fq)
    x_c_int = fq_to_int(x_c)
    x_f_int = fq_to_int(x_f)
    y_c_int = fq_to_int(y_c)
    y_f_int = fq_to_int(y_f)
    sign_bit = 1 if y_f_int > (field_modulus - 1) // 2 else 0
    x_c_bytes = int_to_bytes(x_c_int, 48)
    x_f_bytes = int_to_bytes(x_f_int, 48)
    x_bytes = x_c_bytes + x_f_bytes
    first_byte = x_bytes[0] | 0x80  # compression bit set
    if sign_bit:
        first_byte |= 0x20

    compressed = bytes([first_byte]) + x_bytes[1:]
    return '0x' + compressed.hex()

def decompress_G2_custom(g2_tau_hex: str):
    compressed_bytes = bytes.fromhex(g2_tau_hex[2:])
    assert len(compressed_bytes) == 96

    first_byte = compressed_bytes[0]
    compression_bit = (first_byte & 0x80) != 0
    assert compression_bit, "Compression bit not set"

    sign_bit = (first_byte & 0x20) != 0

    cleared_first_byte = first_byte & 0x1F
    x_bytes = bytes([cleared_first_byte]) + compressed_bytes[1:48]
    x_bytes_im = compressed_bytes[48:96]

    x_c = int.from_bytes(x_bytes, 'big') % field_modulus
    x_f = int.from_bytes(x_bytes_im, 'big') % field_modulus

    x = FQ2([FQ(x_c), FQ(x_f)])

    rhs = x ** 3 + b2

    y = modular_squareroot_in_FQ2(rhs)

    if y is None:
        raise ValueError("No sqrt for y")

    if not isinstance(y, FQ2):
        raise ValueError(f"sqrt_FQ2 did not return FQ2 element but {type(y)}")

    y_re, y_im = y.coeffs
    if isinstance(y_re, int):
        y_re = FQ(y_re)
    if isinstance(y_im, int):
        y_im = FQ(y_im)

    y = FQ2([y_re, y_im])


    if (y.coeffs[1].n > field_modulus // 2) != sign_bit:
        y = FQ2([FQ(-y.coeffs[0].n % field_modulus), FQ(-y.coeffs[1].n % field_modulus)])

    return (x, y, FQ2.one())


# --------------------------------
# KZG core functions
# --------------------------------

def opt_trusted_setup(fx):
    tau = secrets.randbelow(opt_curve_order - 1) + 1
    g1 = opt_G1
    g2 = opt_G2
    degree = len(fx) - 1
    # Generate SRS
    srs = opt_generate_srs(degree, tau)
    if DEBUG_OUTPUT:
        print(f"\nOur SRS on fx {'(first 5 terms out of ' + str(degree+1) + '):' if degree >= 5 else ''}")
        for i, point in enumerate(srs[:min(5, len(srs))]): print(f"g^tau^{i} = ({opt_normalize(point)})")
    g2_tau = opt_multiply(opt_G2, tau)
    tau = None
    return srs, g1, g2, g2_tau

def opt_generate_srs(degree, tau):
    """Generate SRS: [g^tau^0, g^tau^1, ..., g^tau^degree]"""
    return [opt_multiply(opt_G1, pow(tau, i, opt_curve_order)) for i in range(degree + 1)]

def opt_kzg_commit(coeffs, srs):
    """Commit to a polynomial: commitment = g^{f(tau)}"""
    commitment = opt_Z1
    for i, coeff in enumerate(coeffs):
        term = opt_multiply(srs[i], coeff % opt_curve_order)
        commitment = opt_add(commitment, term)
    return commitment

def opt_kzg_prove(fx, z, y, srs):
    """Witness: proof = kzg_commit(f(x)-y)/(x - z)"""
    fx_minus_y = fx[:]
    fx_minus_y[0] = (fx_minus_y[0] - y) % opt_curve_order
    denom = [(-z) % opt_curve_order, 1]
    output = [0] * (len(fx_minus_y) - len(denom) + 1)
    while len(fx_minus_y) >= len(denom):
        lead_coeff = fx_minus_y[-1] * pow(denom[-1], -1, opt_curve_order) % opt_curve_order
        deg_diff = len(fx_minus_y) - len(denom)
        output[deg_diff] = lead_coeff
        for i in range(len(denom)):
            fx_minus_y[i + deg_diff] = (fx_minus_y[i + deg_diff] - lead_coeff * denom[i]) % opt_curve_order
        while fx_minus_y and fx_minus_y[-1] == 0:
            fx_minus_y.pop()
    return opt_kzg_commit(output, srs)

def opt_kzg_verify(commitment, proof, z, y, opt_g1, opt_g2, opt_g2_tau):
    """Verify by checking: e(C−y⋅g1,g2) == e(π,g2^{τ−z})"""
    # Compute commitment - y⋅g1
    left_expr = opt_add(commitment, opt_neg(opt_multiply(opt_g1, y % opt_curve_order)))
    left = opt_final_exponentiate(opt_pairing(opt_g2, left_expr))

    # Compute g2^{τ-z} = g2^τ ⋅ g2^{-z}
    g2_neg_z = opt_multiply(opt_g2, -z % opt_curve_order)
    g2_tau_minus_z = opt_add(opt_g2_tau, g2_neg_z)

    # Compare e(C-y⋅g1,g2) == e(π,g2^{τ-z})
    right = opt_final_exponentiate(opt_pairing(g2_tau_minus_z, proof))

    return left == right

# Multi points batch verification
# - Stack proofs and by using the bilinearity properties of pairings, reduce the expensive pairing to 1 on the LHS
# - Use random scalars r_i to prevent prover from fooling the verifier
def opt_batch_verify_same_poly(commitment, proofs, zs, ys, opt_g1, opt_g2, opt_g2_tau):
    assert len(proofs) == len(zs) == len(ys), "Mismatched lengths"
    n = len(zs)

    # Generate random scalars r_i
    r = [secrets.randbelow(opt_curve_order - 1) + 1 for _ in range(n)]

    # === Left pairing input: ∑ r_i · (C - y_i·G1) ===
    left_expr = opt_Z1
    for i in range(n):
        term = opt_add(commitment, opt_neg(opt_multiply(opt_g1, ys[i] % opt_curve_order)))
        left_expr = opt_add(left_expr, opt_multiply(term, r[i]))

    # Compute e(left_expr, G2)
    left = opt_final_exponentiate(opt_pairing(opt_g2, left_expr))

    # === Right pairing: product over i of e(π_i, g2^{τ - z_i})^{r_i} ===
    product = None
    for i in range(n):
        g2_tau_minus_z = opt_add(opt_g2_tau, opt_multiply(opt_g2, (-zs[i]) % opt_curve_order))
        pair = opt_pairing(g2_tau_minus_z, proofs[i])
        powered = opt_final_exponentiate(pair)
        powered_r = powered ** r[i]
        product = powered_r if product is None else product * powered_r

    return left == product


print("Done loading our KZG implementation.")


Done loading our KZG implementation.


In [172]:
# @title
# Cell 15
# ------------------------------------------------------------
# Code to load public Trusted SRS
# ------------------------------------------------------------
import json

data = json.loads(SRS_str)
g1_hex_list = data["SRS"]["G1Powers"]
g2_hex_list = data["SRS"]["G2Powers"]
g1_points = [decompress_G1(int(s[2:], 16)) for s in data["SRS"]["G1Powers"]]
g2_points = []
for s in data["SRS"]["G2Powers"]:
    hex_str = s[2:]  # Remove '0x'
    if len(hex_str) != 192:
        raise ValueError("Each G2 compressed point must be 96 bytes (192 hex characters)")
    z1 = int(hex_str[:96], 16)
    z2 = int(hex_str[96:], 16)
    g2_points.append(decompress_G2((z1, z2)))


def opt_trusted_setup_with_trusted_srs(fx):
    degree = len(fx) - 1
    if degree >= len(g1_points):
        return opt_trusted_setup(fx) # Use the slow version
    g1 = opt_G1
    g2 = opt_G2
    srs = g1_points[:degree + 1]
    g2_tau = g2_points[1]

    if DEBUG_OUTPUT:
        print(f"\nLoaded SRS (first 5 terms out of {degree+1}):")
        for i, point in enumerate(srs[:min(5, len(srs))]):
            print(f"g^tau^{i} = {opt_normalize(point)}")

    return srs, g1, g2, g2_tau

print("Done loading code for Trusted SRS.")

Done loading code for Trusted SRS.


In [173]:
print("Generate commitment and proof")

hx = blob_to_coeffs(blob_bytes)
srs, g1, g2, g2_tau = opt_trusted_setup_with_trusted_srs(hx)
g2_tau_hex = compress_G2_affine(opt_normalize(g2_tau))
commitment = opt_kzg_commit(hx, srs)
blob_commitment_hex = compress_G1_affine(opt_normalize(commitment))
print("- Commitment \t=",blob_commitment_hex)

# In Ethereum’s protocol, z (the evaluation challenge) is deterministically
# derived in a way not publicly specified (e.g., possibly from the blob data
# or a trusted setup). For simulation, we use a random z."
z = secrets.randbelow(opt_curve_order - 1) + 1
y = evaluate_polynomial(hx, z)

proof = opt_kzg_prove(hx, z, y, srs)
blob_proof_hex = compress_G1_affine(opt_normalize(proof))
print("- Proof \t=", blob_proof_hex)


Generate commitment and proof
Loaded pre-downloaded blob.
- Commitment 	= 0xa66413d6d7ba9615aac6f04ac23886851f8062abee2b50cd9d4c3d55a5d17a3ed51ed38f4c0fd6e742c2f44ab028ea65
- Proof 	= 0x827b491431084071ba2c1160e172a99c5c5730b2f6f4150f463f84e46af03da2456c4aba728f867c102fc0bc2f21cb6f


* #### Submits the blob commitments on-chain, along with a sidecar containing the blob data, proofs, and other related information.

### Validator

* #### Retrieves the blob data and sidecar from the chain.

In [174]:
blob_bytes_validator = blob_bytes
blob_commitment_hex_validator = blob_commitment_hex
blob_proof_hex_validator = blob_proof_hex
print("Done Loading blob, blob commitment, blob proof and g_2^tau.")

Done Loading blob, blob commitment, blob proof and g_2^tau.


* #### Generates the polynomial representation from the blob data.


In [175]:
fx = blob_to_coeffs(blob_bytes_validator)
print("Done turning blob data into polynomial fx.")

Loaded pre-downloaded blob.
Done turning blob data into polynomial fx.


* #### Uses the given or derived challenge points z and evaluations y.

In [176]:
# Retrieve z from the same the evaluation challenge.

z_validator = z # In our case just use the same z
print("z = ",z_validator)

y_validator = evaluate_polynomial(blob_to_coeffs(blob_bytes), z_validator)
print("y is too large to display.")

print("Done generating y with z using the polynomial.")

z =  51905516767214799208853103261812025614314697641697138508284811176627369751565
Loaded pre-downloaded blob.
y is too large to display.
Done generating y with z using the polynomial.


* #### Verifies the blob’s correctness using the commitment, proof, and these values.

In [177]:
commitment_validator = decompress_G1(int(blob_commitment_hex_validator[2:], 16))
print("Decompress commitment:", commitment_validator)

proof_validator = decompress_G1(int(blob_proof_hex_validator[2:], 16))
print("Decompress proof:", proof_validator)

g2_tau_validator = decompress_G2_custom(g2_tau_hex)

valid = opt_kzg_verify(commitment_validator, proof_validator, z_validator, y_validator, opt_G1, opt_G2, g2_tau_validator)
print(f"\nThe commitment is valid: {valid}")



Decompress commitment: (983653803621734741406587933903683989828984047129987230612533744849316881492710590744332995305204262259462358231653, 2205914201314182185744244984752011873548107904797165622855729378437396444594206775593490199594564087008631086889847, 1)
Decompress proof: (381950711101261087727054183844606496621165939359143236547599159832347364716619009910203447706089207440071821740911, 198109135237623027182972030576989270314726433575310346204804374076310199303662538305408029343618640463797676921586, 1)

The commitment is valid: True


## Conclusion

By using KZG commitments in Ethereum, the benefits are:

1. Constant-size commitments and proofs
  * KZG commitments produce a fixed-size (48 bytes) commitment and small proofs regardless of the data size. This keeps on-chain storage and gas costs low even for very large data blobs.

2. Supports succinct proofs for validity and availability
  * KZG proofs allow validators to efficiently confirm that the data blobs are correct and available, improving trust without sacrificing performance.

3. Well-studied cryptographic foundation
  * KZG commitments rely on pairing-based cryptography, with strong security assumptions, which is suitable for blockchain use.